### Leer el dataframe guardado

In [1]:
import pandas as pd

def csv_to_dataframe(file_name):
    return pd.read_csv(file_name)

In [3]:
# dataframe = csv_to_dataframe('./files/warped_terrain_sdf.csv')

In [4]:
# dataframe

,x,y,z,density,distance
0,0.00,0.00,0.00,0.000000,0.070711
1,0.00,0.00,0.05,0.032586,0.050000
2,0.00,0.00,0.10,0.061833,-0.050000
3,0.00,0.00,0.15,0.082446,-0.100000
4,0.00,0.00,0.20,0.097867,-0.150000
...,...,...,...,...,...
7995,0.95,0.95,0.75,0.377797,0.860233
7996,0.95,0.95,0.80,0.366430,0.890225
7997,0.95,0.95,0.85,0.352210,0.921954
7998,0.95,0.95,0.90,0.334525,0.955249


### 5. Funcion para crear esferas

In [ ]:
# len(dataframe)

In [ ]:
# dataframe

In [2]:
from math import sqrt

def distanceSquared3D(point, other):
    return sqrt((point[0]-other[0])**2 + (point[1]-other[1])**2 + (point[2]-other[2])**2)

def sphereIsInSphere(spheres, point, radio):
    for sphere in spheres: #sphere = [point, radio]
        distance = distanceSquared3D(point, sphere[0])
        if distance < sphere[1]:
            return True, 0
        elif distance < sphere[1]+radio:
            return True, distance-sphere[1]
    return False, radio

# take second element for sort
def takeSecond(elem):
    return elem[1]

def makeSpheres(dataframe, max_spheres): #dataframe = (x,y,z,density,distance)
    coordinates_spheres = [] #[(Point,r)]
    spheres_back = []
    
    for i in range(len(dataframe)):
        #print(i)
        distance = dataframe.iloc[i].distance
        if (distance < 0 and len(coordinates_spheres) < max_spheres):
            #print()
            coord = [dataframe.iloc[i].y, dataframe.iloc[i].x, dataframe.iloc[i].z]
            isInSphere, posibleRadio = sphereIsInSphere(coordinates_spheres, coord, -distance)
            if(not isInSphere):
                coordinates_spheres.append([coord, -distance])
            elif posibleRadio > 0:
                spheres_back.append([coord, posibleRadio])
                
    if len(coordinates_spheres) < max_spheres:
        spheres_back.sort(key=takeSecond, reverse=True)
        
    while len(coordinates_spheres) < max_spheres and i < len(spheres_back):
        coordinates_spheres.append([spheres_back[i]])
        i += 1
        
    return coordinates_spheres

def generateSpheres(dataframe, max_spheres):
    sort_dataframe = dataframe.sort_values(["distance"])#, ascending=False)
    spheresPoints = makeSpheres(sort_dataframe, max_spheres)
    # plot(circles, use_brightness_level, do_loop)
    return spheresPoints

In [3]:
def generateShaderWithSpheres(spheres, idx):    
    f = "float soil{0}SDF(vec3 inputPoint) {{\nreturn ".format(idx)
    sphere = spheres[0]
    x_max=1
    y_max=1
    z_max=1
    r_max=1
    prev = "sphereSDF(inputPoint, vec3({0}, {1}, {2}), {3})".format(sphere[0][0]/x_max, sphere[0][1]/y_max, sphere[0][2]/z_max, float(sphere[1]/r_max))
    if (len(spheres) > 1):
        for i in range(1,len(spheres)):
            sphere = spheres[i]
            opt = "smoothUnion(sphereSDF(inputPoint, vec3({0}, {1}, {2}), {3}), {4})".format(sphere[0][0]/x_max, sphere[0][1]/y_max, sphere[0][2]/z_max, float(sphere[1]/r_max), prev)
            prev = opt
        f += opt
    else:
        f += prev
    f += ";\n}\n\n"
    return f

In [ ]:
# finalShaderString = "float smoothUnion(float a, float b) {\n    float k = 0.1;\n float h = max( k-abs(a-b), 0.0 )/k;\nreturn min( a, b ) - h*h*k*(1.0/4.0); }\nfloat sphereSDF(vec3 p, vec3 pos, float r) { return length(pos - p) - r; }\n\n"

# for i in range(len(internalValues)):
#     dataframe = csv_to_dataframe(r'./files/{}_warped_terrain_sdf.csv'.format(i))
#     max_number_of_spheres = 50
#     spheres = generateSpheres(dataframe, max_number_of_spheres)
#     finalShaderString = finalShaderString + generateShaderWithSpheres(spheres, i)

# print (finalShaderString)

In [ ]:
spheres = generateSpheres(dataframe, 50)

In [ ]:
spheres

### 5. Funcion para crear cubos

In [5]:
coordinates_cubes = [] #[(Point,L)]

In [4]:
def vertexInCube(vertex, X_min, X_max, Y_min, Y_max, Z_min, Z_max):
    X = vertex[0]
    Y = vertex[1]
    Z = vertex[2]
    
    return X_min <= X <= X_max and Y_min <= Y <= Y_max  and Z_min <= Z <= Z_max
    
    
def getXYZMax(point, side):
    X = point[0]
    Y = point[1]
    Z = point[2]

    X_max = X + side
    X_min = X - side

    Y_max = Y + side
    Y_min = Y - side

    Z_max = Z + side
    Z_min = Y - side
    
    return X_min, X_max, Y_min, Y_max, Z_min, Z_max
    
    
def cubeInCube(cube1, cube2): # cube = [(point), side]
    '''Return if part of cube1 is inside cube2'''
    point = cube2[0]
    side = cube2[1]
    X_min, X_max, Y_min, Y_max, Z_min, Z_max = getXYZMax(point, side)
    
    
    #######################################################

    point = cube1[0]
    side = cube1[1]
    X1_min, X1_max, Y1_min, Y1_max, Z1_min, Z1_max = getXYZMax(point, side)
    
    vertex1 = (X1_max, Y1_min, Z1_max)
    vertex2 = (X1_max, Y1_min, Z1_min)
    vertex3 = (X1_max, Y1_max, Z1_max)
    vertex4 = (X1_max, Y1_max, Z1_min)
    vertex5 = (X1_min, Y1_max, Z1_max)
    vertex6 = (X1_min, Y1_max, Z1_min)
    vertex7 = (X1_min, Y1_min, Z1_max)
    vertex8 = (X1_min, Y1_min, Z1_min)
    
    vertexes = [vertex1, vertex2, vertex3, vertex4, vertex5, vertex6, vertex7, vertex8]
    
    for vertex in vertexes:
        if(vertexInCube(vertex, X_min, X_max, Y_min, Y_max, Z_min, Z_max)):
            return True
    return False


def cubeIsInsideCube(cubes, point, side):
    originalCube = [point, side]
    for cube in cubes:
        if(cubeInCube(originalCube, cube) or cubeInCube(cube, originalCube)):
            return True
    return False



In [5]:
def makeCubes(dataframe, max_cubes): #dataframe = (x,y,z,density,distance)
    coordinates_cubes = [] #[(Point,r)]
    cubes_back = []
    
    for i in range(len(dataframe)):
        #print(i)
        distance = dataframe.iloc[i].distance
        if (distance < 0 and len(coordinates_cubes) < max_cubes):
            #print()
            coord = [dataframe.iloc[i].y, dataframe.iloc[i].x, dataframe.iloc[i].z]
            isInCube = cubeIsInsideCube(coordinates_cubes, coord, -distance)
            
            # isInSphere, posibleRadio = sphereIsInSphere(coordinates_spheres, coord, -distance)
            if(not isInCube):
                coordinates_cubes.append([coord, -distance])
                
# TODO: AGREGAR MAS CUBOS COMO EN LAS ESFERAS EN CASO DE NO HABER LLEGADO AL NUMERO MAXIMO DE ESFERAS
#             elif posibleRadio > 0:
#                 spheres_back.append([coord, posibleRadio])
                
#     if len(coordinates_cubes) < max_cubes:
#         cubes_back.sort(key=takeSecond, reverse=True)
        
#     while len(coordinates_cubes) < max_cubes and i < len(cubes_back):
#         coordinates_cubes.append([cubes_back[i]])
#         i += 1
        
    return coordinates_cubes

def generateCubes(dataframe, max_cubes):
    sort_dataframe = dataframe.sort_values(["distance"])#, ascending=False)
    cubesPoints = makeCubes(sort_dataframe, max_cubes)
    return cubesPoints

In [10]:
# cubes = generateCubes(dataframe, 50)

In [11]:
# cubes

[[[0.95, 0.15000000000000002, 0.0], 0.1870828693386971],
 [[0.6000000000000001, 0.05, 0.15000000000000002], 0.15000000000000008],
 [[0.0, 0.0, 0.2], 0.15000000000000002],
 [[0.95, 0.1, 0.2], 0.15],
 [[0.95, 0.0, 0.25], 0.14142135623730948],
 [[0.6000000000000001, 0.0, 0.2], 0.12247448713915896],
 [[0.30000000000000004, 0.0, 0.2], 0.11180339887498948],
 [[0.1, 0.25, 0.0], 0.0866025403784439],
 [[0.25, 0.2, 0.0], 0.08660254037844385],
 [[0.05, 0.25, 0.05], 0.07071067811865478],
 [[0.0, 0.25, 0.05], 0.07071067811865478],
 [[0.35000000000000003, 0.25, 0.0], 0.07071067811865478],
 [[0.1, 0.25, 0.05], 0.07071067811865478],
 [[0.30000000000000004, 0.25, 0.0], 0.07071067811865478],
 [[0.05, 0.30000000000000004, 0.0], 0.07071067811865475],
 [[0.25, 0.25, 0.0], 0.07071067811865475],
 [[0.35000000000000003, 0.2, 0.05], 0.07071067811865475],
 [[0.15000000000000002, 0.25, 0.0], 0.07071067811865475],
 [[0.30000000000000004, 0.2, 0.05], 0.07071067811865475],
 [[0.25, 0.2, 0.05], 0.07071067811865474],

In [12]:
# len(cubes)

50

### 6. Funcion para generar shader

Siendo
c = centro del eje
p = punto donde esta el cubo
s = tama;o del lado

##### Sacado de https://www.alanzucconi.com/2016/07/01/signed-distance-functions/
```
float sdf_box (float3 p, float3 c, float3 s){
    float x = max
    (   p.x - c.x - float3(s.x / 2., 0, 0),
        c.x - p.x - float3(s.x / 2., 0, 0)
    );
    
    float y = max
    (   p.y - c.y - float3(s.y / 2., 0, 0),
        c.y - p.y - float3(s.y / 2., 0, 0)
    );
    
    float z = max
    (   p.z - c.z - float3(s.z / 2., 0, 0),
        c.z - p.z - float3(s.z / 2., 0, 0)
    );
    
    float d = x;
    d = max(d,y);
    d = max(d,z);
    return d;
}
```

##### Esto es lo mismo que
```
float vmax(float3 v){
    return max(max(v.x, v.y), v.z);
}

float sdf_boxcheap(float3 p, float3 c, float3 s){
    return vmax(abs(p-c) - s);
}
```

In [7]:
def generateShaderWithCubes(cubes, idx):
#     f = """float cubeSDF(vec3 p) {
#     vec3 d = abs(p) - vec3(1.0, 1.0, 1.0);
#     return min(max(d.x,max(d.y,d.z)),0.0) + length(max(d,0.0));
# }"""
    functionName = "cubeSDFFinal{0}".format(idx)
    f = """float vmax(vec3 v){
    \n     return max(max(v.x, v.y), v.z);
    \n}
    \n
    \nfloat sdf_boxcheap(vec3 c, vec3 p, vec3 s){
    \n     return vmax(abs(p-c) - s);
    \n}
    \n
    \nfloat """+functionName+"(vec3 inputPoint) {\nreturn "

    cube = cubes[0]
    x_max=1
    y_max=1
    z_max=1
    r_max=1
    prev = "sdf_boxcheap(inputPoint, vec3({0}, {1}, {2}), vec3({3}, {3}, {3}))".format(cube[0][0]/x_max, cube[0][1]/y_max, cube[0][2]/z_max, float(cube[1]/r_max))
    if (len(cubes) > 1):
        for i in range(1,len(cubes)):
            cube = cubes[i]
            opt = "smoothUnion(sdf_boxcheap(inputPoint, vec3({0}, {1}, {2}), vec3({3}, {3}, {3})), {4})".format(cube[0][0]/x_max, cube[0][1]/y_max, cube[0][2]/z_max, float(cube[1]/r_max), prev)
            prev = opt
        f += opt
    else:
        f += prev
    f += ";\n}"
    print(f)

In [17]:
# generateShaderWithCubes(cubes)

float vmax(vec3 v){
    
     return max(max(v.x, v.y), v.z);
    
}
    

    
float sdf_boxcheap(vec3 c, vec3 p, vec3 s){
    
     return vmax(abs(p-c) - s);
    
}
    

    
float cubeSDFFinal(vec3 inputPoint) {
return smoothUnion(sdf_boxcheap(inputPoint, vec3(0.25, 0.25, 0.05), vec3(0.04999999999999999, 0.04999999999999999, 0.04999999999999999)), smoothUnion(sdf_boxcheap(inputPoint, vec3(0.2, 0.30000000000000004, 0.0), vec3(0.04999999999999999, 0.04999999999999999, 0.04999999999999999)), smoothUnion(sdf_boxcheap(inputPoint, vec3(0.7000000000000001, 0.35000000000000003, 0.0), vec3(0.04999999999999999, 0.04999999999999999, 0.04999999999999999)), smoothUnion(sdf_boxcheap(inputPoint, vec3(0.25, 0.30000000000000004, 0.0), vec3(0.04999999999999999, 0.04999999999999999, 0.04999999999999999)), smoothUnion(sdf_boxcheap(inputPoint, vec3(0.30000000000000004, 0.30000000000000004, 0.0), vec3(0.04999999999999999, 0.04999999999999999, 0.04999999999999999)), smoothUnion(sdf_boxcheap(inputPoint, 

# Cosas a escribir en el shader

### Definicion de variables

In [97]:
internalValues = [[0.00,0.081], [0.082,0.17], [0.18,0.27], [0.28,0.4]]

In [98]:
shape = "sphere"

In [100]:
if shape == "cube":
    baseNameFunctions = "cubeSDFFinal{0}"
else:
    baseNameFunctions = "soil{0}SDF"

### Funciones para generar la escena total

In [94]:
def generateFunctionsToBuildTotalScene(internalValues, shape="sphere", max_number_of_shapes=50):
    finalShaderString = "\nfloat smoothUnion(float a, float b) {\n    float k = 0.1;\n    float h = max( k-abs(a-b), 0.0 )/k;\n    return min( a, b ) - h*h*k*(1.0/4.0); \n}\n"
    sphereShaderFunction = "\nfloat sphereSDF(vec3 p, vec3 pos, float r) {\n    return length(pos - p) - r; \n}\n"
    cubeShaderFunction = """\nfloat vmax(vec3 v){\n     return max(max(v.x, v.y), v.z);\n}
        \nfloat sdf_boxcheap(vec3 c, vec3 p, vec3 s){\n     return vmax(abs(p-c) - s);\n}\n"""
    
    finalShaderString += sphereShaderFunction
    finalShaderString += cubeShaderFunction
    finalShaderString += "\n"
        
    for i in range(len(internalValues)):
        dataframe = csv_to_dataframe(r'./files/{}_warped_terrain_sdf.csv'.format(i))

        if shape == "cube":
            cubes = generateCubes(dataframe, max_number_of_shapes)
            functions = generateShaderWithCubes(cubes, i)
        else: # default shape is sphere
            spheres = generateSpheres(dataframe, max_number_of_shapes)
            functions = generateShaderWithSpheres(spheres, i)
        

        finalShaderString = finalShaderString + functions

    return finalShaderString

In [95]:
internalValues = [[0.00,0.081], [0.082,0.17], [0.18,0.27], [0.28,0.4]]
functionsToBuildTotalScene = generateFunctionsToBuildTotalScene(internalValues, shape)

In [96]:
print(functionsToBuildTotalScene)


float smoothUnion(float a, float b) {
    float k = 0.1;
    float h = max( k-abs(a-b), 0.0 )/k;
    return min( a, b ) - h*h*k*(1.0/4.0); 
}

float sphereSDF(vec3 p, vec3 pos, float r) {
    return length(pos - p) - r; 
}

float vmax(vec3 v){
     return max(max(v.x, v.y), v.z);
}
        
float sdf_boxcheap(vec3 c, vec3 p, vec3 s){
     return vmax(abs(p-c) - s);
}

float soil0SDF(vec3 inputPoint) {
return smoothUnion(sphereSDF(inputPoint, vec3(0.5, 0.1, 0.0), 0.04999999999999999), smoothUnion(sphereSDF(inputPoint, vec3(0.8, 0.0, 0.25), 0.04999999999999999), smoothUnion(sphereSDF(inputPoint, vec3(0.8, 0.1, 0.15000000000000002), 0.04999999999999999), smoothUnion(sphereSDF(inputPoint, vec3(0.7000000000000001, 0.0, 0.2), 0.05), smoothUnion(sphereSDF(inputPoint, vec3(0.5, 0.0, 0.1), 0.07071067811865475), smoothUnion(sphereSDF(inputPoint, vec3(0.95, 0.0, 0.1), 0.1118033988749895), smoothUnion(sphereSDF(inputPoint, vec3(0.0, 0.0, 0.0), 0.12247448713915893), smoothUnion(sphereSDF(inputPoin

### TOTAL SceneSDF Function

In [24]:
def generateSceneSDFFunction(baseNameOfFunctionWithParameter="soil{0}SDF", internalValues=[]):
    
    sceneSDFFunction = """\n/**
 \n* Signed distance function describing the scene.
 \n* 
 \n* Absolute value of the return value indicates the distance to the surface.
 \n* Sign indicates whether the point is inside or outside the surface,
 \n* negative indicating inside.
 \n*/
 \nfloat sceneSDF(vec3 samplePoint) {
\n    float dist = max("""

    amountOfInternalValues = len(internalValues)

    if amountOfInternalValues <= 1:
        sceneSDFFunction += baseNameOfFunctionWithParameter.format("") + "(samplePoint)"
    else:
        pre = baseNameOfFunctionWithParameter.format(0) + "(samplePoint)"
        for i in range(1, amountOfInternalValues):
            nameOfFunction = baseNameOfFunctionWithParameter.format(i) + "(samplePoint)"
            minFunction = "min({0}, {1})".format(pre, nameOfFunction)
            pre = minFunction
        sceneSDFFunction += minFunction
        
    sceneSDFFunction += """, cubeSDF(samplePoint));
    \n    dist = max(dist, -sphereSDF(samplePoint, vec3(sin(iTime), cos(iTime),sin(iTime)), sin(0.23*3.0*iTime)));
    \n
    \n    return dist;
    \n}"""
    
    return sceneSDFFunction

In [25]:
sceneFunction = generateSceneSDFFunction(baseNameFunctions, internalValues)

In [26]:
print(sceneFunction)


/**
 
* Signed distance function describing the scene.
 
* 
 
* Absolute value of the return value indicates the distance to the surface.
 
* Sign indicates whether the point is inside or outside the surface,
 
* negative indicating inside.
 
*/
 
float sceneSDF(vec3 samplePoint) {

    float dist = max(min(min(min(soil0SDF(samplePoint), soil1SDF(samplePoint)), soil2SDF(samplePoint)), soil3SDF(samplePoint)), cubeSDF(samplePoint));
    
    dist = max(dist, -sphereSDF(samplePoint, vec3(sin(iTime), cos(iTime),sin(iTime)), sin(0.23*3.0*iTime)));
    

    
    return dist;
    
}


### Main function

In [27]:
color1 = "vec3(72.0/255.0,209.0/255.0,204.0/255.0)"
color2 = "vec3(124.0/255.0,252.0/255.0,0.0)"
color3 = "vec3(1.0,1.0,161.0/255.0)"
color4 = "vec3(152.0/255.0,116.0/255.0,86.0/255.0)"
colorsOfStrates = [color1, color2, color3, color4]

In [50]:
def generateMainFunction(baseNameOfFunctionWithParameter="soil{0}SDF", internalValues=[]):
    mainFunction = """\nvoid mainImage( out vec4 fragColor, in vec2 fragCoord )
\n{
\n    float angle = iTime*0.5;
\n    mat3 rotation = mat3(
\n            vec3(cos(angle), 0.0, -sin(angle)),
\n            vec3(0.0, 1.0, 0.0),
\n            vec3(sin(angle), 0.0, cos(angle))
\n        );
\n    vec3 dir = normalize(rotation*rayDirection(45.0, iResolution.xy, fragCoord));
\n    vec3 eye = rotation*vec3(0.0,0.0,10.0);
\n
\n    float dist = shortestDistanceToSurface(eye, dir, MIN_DIST, MAX_DIST);
\n    
\n    if (dist > MAX_DIST - EPSILON) {
\n        // Didn't hit anything
\n        fragColor = vec4(0.0, 0.0, 0.0, 0.0);
\n		return;
\n    }
\n
\n    // The closest point on the surface to the eyepoint along the view ray
\n    vec3 p = eye + dir * dist;
\n
\n    vec3 K_d = vec3(0.7, 0.7, 0.7);
\n    """
    
    amountOfInternalValues = len(internalValues)
    totalMinFunction = "\n    float minSdf = "
    if amountOfInternalValues <= 1:
        variableName = baseNameOfFunctionWithParameter.format("")
        partMinFunction = variableName+"(p)"
        
        K_d_vectorDefinition = "\n    K_d = "+colorsOfStrates[0]
    else:
        partMinFunction = ""
        preMinFunction = "sdf{0}".format(0)
        K_d_vectorDefinition = "\n    "
            
        for i in range(amountOfInternalValues):
            variableName = "sdf{0}".format(i)
            
            mainFunction += "\n    float "+variableName+" = "+baseNameOfFunctionWithParameter.format(i)+"(p);"
            
            
            partMinFunction = "min({0}, {1})".format(preMinFunction, variableName)
            preMinFunction = partMinFunction
            
            K_d_vectorDefinition += "\n    if (abs({0} - minSdf) < 0.1)".format(variableName)
            K_d_vectorDefinition += " {"
            K_d_vectorDefinition += "\n        K_d = {0};".format(colorsOfStrates[i])
            K_d_vectorDefinition += "\n    }"
            
    
    totalMinFunction += partMinFunction + ";"

    mainFunction += totalMinFunction
    mainFunction += K_d_vectorDefinition
    
    mainFunction += """\n
\n    vec3 K_a = vec3(0.2, 0.2, 0.2);
\n    vec3 K_s = vec3(1.0, 1.0, 1.0);
\n    float shininess = 1.0;
\n    
\n    vec3 color = K_d;//phongIllumination(K_a, K_d, K_s, shininess, p, eye);
\n    
\n    fragColor = vec4(color * length(p), 1.0);
\n}
\n
\nvoid main() {
\n   mainImage(gl_FragColor, gl_FragCoord.xy);
\n}"""
    
    return mainFunction

In [51]:
mainFunction = generateMainFunction(baseNameFunctions, internalValues)

In [52]:
print(mainFunction)

void mainImage( out vec4 fragColor, in vec2 fragCoord )

{

    float angle = iTime*0.5;

    mat3 rotation = mat3(

            vec3(cos(angle), 0.0, -sin(angle)),

            vec3(0.0, 1.0, 0.0),

            vec3(sin(angle), 0.0, cos(angle))

        );

    vec3 dir = normalize(rotation*rayDirection(45.0, iResolution.xy, fragCoord));

    vec3 eye = rotation*vec3(0.0,0.0,10.0);



    float dist = shortestDistanceToSurface(eye, dir, MIN_DIST, MAX_DIST);

    

    if (dist > MAX_DIST - EPSILON) {

        // Didn't hit anything

        fragColor = vec4(0.0, 0.0, 0.0, 0.0);

		return;

    }



    // The closest point on the surface to the eyepoint along the view ray

    vec3 p = eye + dir * dist;



    vec3 K_d = vec3(0.7, 0.7, 0.7);

    
    float sdf0 = soil0SDF(p);
    float sdf1 = soil1SDF(p);
    float sdf2 = soil2SDF(p);
    float sdf3 = soil3SDF(p);
    float minSdf = min(min(min(min(sdf0, sdf0), sdf1), sdf2), sdf3);
    
    if (abs(sdf0 - minSdf) < 0.1) {
        K_d

# Crear archivo shader

In [56]:
import os
# remove file if it exists
shaderFileName = './files/shader/raymarcher.glsl'
try:
    os.remove(shaderFileName)
except OSError:
    pass


# import module
import shutil
  
# use copyfile() --> copy basic shader
shutil.copyfile('./files/shader/basic_shader.glsl',shaderFileName)

# write scene function
text_file = open(shaderFileName, "a")

text_file.write(functionsToBuildTotalScene)
text_file.write(sceneFunction)
text_file.write(mainFunction)

text_file.close()